In [1]:
#!pip install langdetect
#!pip install tqdm
#nltk.download('wordnet')
#nltk.download('stopwords')
#!pip install ipywidgets
#!jupyter nbextension enable --py widgetsnbextension

#!pip install dash
#!pip install dash-renderer
#!pip install dash_html_components
#!pip install dash_core_components
#!pip install jupyter_dash

In [2]:
# Imports 
from pprint import pprint
from tqdm.notebook import tqdm
from langdetect import DetectorFactory, detect
import pandas as pd
import numpy as np
import seaborn as sns
import re
import io
import string
import plotly.express as px
import unicodedata
import ipywidgets as widgets
from ipywidgets import interact, IntText, widgets, FileUpload
import nltk
wn = nltk.WordNetLemmatizer()

# Plotly dash Imports
import dash
import dash_core_components as dcc
import dash_html_components as html
import jupyter_dash

In [ ]:
import base64
import datetime
import io
import jupyter_dash

from jupyter_dash import JupyterDash
from dash import no_update
import dash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html
import dash_table

import pandas as pd


external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    dcc.Upload(
        id='upload-data',
        children=html.Div([
            'Drag and Drop or ',
            html.A('Select Files')
        ]),
        style={
            'width': '100%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px'
        },
        # Allow multiple files to be uploaded
        multiple=True
    ),
    html.Div(id='output-div'),
    html.Div(id='output-data-upload'),
    
])


def parse_contents(contents, filename, date):
    content_type, content_string = contents.split(',')

    decoded = base64.b64decode(content_string)
    try:
        if 'csv' in filename:
            # Assume that the user uploaded a CSV file
            df = pd.read_csv(
                io.StringIO(decoded.decode('utf-8')))
        elif 'xls' in filename:
            # Assume that the user uploaded an excel file
            df = pd.read_excel(io.BytesIO(decoded))
    except Exception as e:
        print(e)
        return html.Div([
            'There was an error processing this file.'
        ])

    return html.Div([
        html.H5(filename),
        html.H6(datetime.datetime.fromtimestamp(date)),
        html.P("Select Column"),
        dcc.Dropdown(id='column-data',
                     options=[{'label':x, 'value':x} for x in df.columns]),
        html.Button(id="submit-button", children="Create Graph"),
        
        html.Hr(),

        dash_table.DataTable(
            data=df.to_dict('records'),
            columns=[{'name': i, 'id': i} for i in df.columns],
            page_size=15
        ),
        dcc.Store(id='stored-data', data=df.to_dict('records')),

        html.Hr(),  # horizontal line

        # For debugging, display the raw contents provided by the web browser
        html.Div('Raw Content'),
        html.Pre(contents[0:200] + '...', style={
            'whiteSpace': 'pre-wrap',
            'wordBreak': 'break-all'
        })
    ])

@app.callback(Output('output-data-upload', 'children'),
              Input('upload-data', 'contents'),
              State('upload-data', 'filename'),
              State('upload-data', 'last_modified'))
def update_output(list_of_contents, list_of_names, list_of_dates):
    if list_of_contents is not None:
        children = [
            parse_contents(c, n, d) for c, n, d in
            zip(list_of_contents, list_of_names, list_of_dates)]
        return children


@app.callback(Output('output-div', 'children'),
              Input('submit-button','n_clicks'),
              State('stored-data','data'),
              State('column-data','value'))
def make_graphs(n, data, key_data):
    if n is None:
        return no_update
    else:
        #
            df_covid = pd.DataFrame(columns=['abstract'])
            data2 = pd.DataFrame(data)
            df_covid['abstract'] = data2[key_data]
            #df_covid['abstract'] = data2.abstract
            
            #df_covid
            df_covid.drop_duplicates(['abstract'], inplace=True)
            df_covid.dropna(inplace=True)
            #df_covid.info()
            #Dropping non-English articles

            DetectorFactory.seed = 0

            # hold label - language
            languages = []

            # go through each text
            for ii in tqdm(range(0,len(df_covid))):
                # split by space into list, take the first x intex, join with space
                text = df_covid.iloc[ii]['abstract'].split(" ")

                lang = "en"
                try:
                    if len(text) > 50:
                        lang = detect(" ".join(text[:50]))
                    elif len(text) > 0:
                        lang = detect(" ".join(text[:len(text)]))
                # ught... beginning of the document was not in a good format
                except Exception as e:
                    all_words = set(text)
                    try:
                        lang = detect(" ".join(all_words))

                    except Exception as e:        
                        lang = "unknown"
                        pass

                # get the language    
                languages.append(lang)
            # Number of articles in each language

            languages_dict = {}
            for lang in set(languages):
                languages_dict[lang] = languages.count(lang)

            print("Total: {}\n".format(len(languages)))
            pprint(languages_dict)
            # Dropping non english articles

            df_covid['language'] = languages
            df_covid = df_covid[df_covid['language'] == 'en'] 
            #df_covid.info()
            df_covid = df_covid.drop(['language'], axis = 1) 
            #df_covid.head()
            df_covid.to_csv('modified.csv')
            
            dataset = pd.read_csv('modified.csv')
            #dataset.head()
            dataset.columns = ['label','abstract']
            #dataset.head()
            dataset = dataset.drop(columns = 'label')

            def remove_punctuation(txt):
                txt_nopunct = "".join([c for c in txt if c not in string.punctuation])
                return txt_nopunct

            dataset['abstract_clean'] = dataset['abstract'].apply(lambda x: remove_punctuation(x))
            #dataset.head()
            
            import re
            def tokenize(txt):
                tokens = re.split('\W+', txt)
                return tokens

            dataset['abstract_clean_tokenized'] = dataset['abstract_clean'].apply(lambda x: tokenize(x.lower()))
            #dataset.head()

            dataset = dataset.drop(columns = ['abstract'])
            dataset = dataset.drop(columns = ['abstract_clean'])
            #dataset.head()
            def lemmatization(token_text):
                text = [wn.lemmatize(word) for word in token_text]
                return text

            dataset['abs_cln_lemm'] = dataset['abstract_clean_tokenized'].apply(lambda x : lemmatization(x))
            dataset.head()

            #Remove stop words
            import nltk
            stopwords = nltk.corpus.stopwords.words('english')

            def remove_stopwords(txt_tokenized):
                txt_clean = [word for word in txt_tokenized if word not in stopwords]
                return txt_clean

            dataset['abs_no_sw'] = dataset['abs_cln_lemm'].apply(lambda x: remove_stopwords(x))
            #dataset.head()

            dataset = dataset.drop(columns = ['abstract_clean_tokenized'])
            dataset = dataset.drop(columns = ['abs_cln_lemm'])
            #dataset.head()

            dataset.to_csv('modified_cleaned.csv')
            # Custom word removal
            new_words = ('2019','95','ci','patient', 'mental', 'health', 'risk', 'factor', 'formula', 'see', 'text', 'polymerase', 'chain', 'reaction' ,'supplementary',
                         'material', 'available' ,'intensive', 'care', 'unit', 'contains', 'supplementary', 
                        'ha', 'acute', 'study', 'wa', '2', '19', 'white', 'blood', 'cell', 'middle', 'east',
                         'aim', 'sars', 'cov', 'electronic', 'online', 'version', 'article',
                         'access', 'service', 'among', 'waste', 'collector', 'routine', 'treatment', 'combined', 'chinese', 'herbal', 
                        'liaison', 'sarscov2', 's1s2', 'igg', 'assay', 'mechanical', 'ventilation', 's1', 'n', 'p', '0001')

            def custom_word_removal(txt_tokenized):
                txt_clean = [word for word in txt_tokenized if word not in new_words]
                return txt_clean

            dataset['abstract_keywords'] = dataset['abs_no_sw'].apply(lambda x: custom_word_removal(x))
            #dataset.head()

            dataset = dataset.drop(columns = ['abs_no_sw'])
            #dataset.head()

            dataset.to_csv('abstract_keywords.csv')
            df_ngram = pd.read_csv('abstract_keywords.csv', usecols=[1])
            
            def basic_clean(text):

              text = (unicodedata.normalize('NFKD', text)
                .encode('ascii', 'ignore')
                .decode('utf-8', 'ignore')
                .lower())
              words = re.sub(r'[^\w\s]', '', text).split()
              return words
            words = basic_clean(''.join(str(df_ngram['abstract_keywords'].tolist())))
            bigrams_series = (pd.Series(nltk.ngrams(words, 2)).value_counts())[:12]
            bigrams_series.to_csv('ram.csv')
            bigrams_series.sort_values().plot.barh(color='blue', width=.9, figsize=(12, 8))
            bigrams_series = (pd.Series(nltk.ngrams(words, 3)).value_counts())[:12]
            bigrams_series.to_csv('tram.csv')
            data1=pd.read_csv('tram.csv')
            data1.columns=['keywords','values']
            data1.dropna(inplace = True)
            new = data1["keywords"].str.split(", ", n = 2, expand = True)
            data1["First keywords"]= new[0]
            data1["middle keywords"]= new[1]
            data1["last keywoprds"]= new[2]
            data1.drop(columns =["keywords"], inplace = True)
            print (data1)
            data1.to_csv('new.csv')
            s = open('new.csv','r').read()

            chars = ('(',')') # etc
            for c in chars:
              s = ''.join( s.split(c) )

            out_file = open('new.csv','w')
            out_file.write(s)
            out_file.close()
            data3=pd.read_csv('new.csv')
            sunburst_fig=px.sunburst(data3,path=['First keywords','middle keywords','last keywoprds'],values="values")
            #sunburst_fig = px.bar(data, x=key_data, y=y_data)
            # print(data)
            return dcc.Graph(figure=sunburst_fig)


if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Jun/2021 19:58:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2021 19:58:07] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2021 19:58:07] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2021 19:58:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2021 19:58:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2021 19:58:22] "GET /_dash-component-suites/dash_table/async-highlight.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2021 19:58:22] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [22/Jun/2021 19:58:22] "GET /_dash-component-suites/dash_table/async-table.js HTTP/1.1" 200 -



Total: 409

{'en': 405, 'es': 2, 'fr': 2}
    values  First keywords middle keywords  last keywoprds
0       52       ('severe'   'respiratory'     'syndrome')
1       52  ('coronavirus'       'disease'      'covid19')
2       41  ('respiratory'      'syndrome'  'coronavirus')
3       12      ('disease'       'covid19'     'pandemic')
4        8        ('novel'   'coronavirus'      'disease')
5        8     ('distress'      'syndrome'         'ards')
6        8  ('respiratory'      'distress'     'syndrome')
7        8       ('caused'        'severe'  'respiratory')
8        7      ('covid19'       'mhealth'         'tool')
9        7  ('respiratory'         'tract'    'infection')
10       6     ('adjusted'          'odds'        'ratio')
11       6        ('basic'  'reproduction'       'number')


127.0.0.1 - - [22/Jun/2021 19:58:59] "POST /_dash-update-component HTTP/1.1" 200 -



Total: 521

{'af': 15,
 'ca': 17,
 'cs': 17,
 'cy': 69,
 'da': 16,
 'de': 4,
 'en': 11,
 'es': 4,
 'et': 5,
 'fi': 8,
 'fr': 10,
 'hr': 22,
 'hu': 18,
 'id': 3,
 'it': 5,
 'lt': 5,
 'lv': 10,
 'nl': 9,
 'no': 5,
 'pl': 61,
 'pt': 13,
 'ro': 10,
 'sk': 25,
 'sl': 22,
 'so': 60,
 'sq': 35,
 'sv': 9,
 'sw': 12,
 'tl': 9,
 'tr': 3,
 'vi': 9}
   values First keywords middle keywords last keywoprds
0       1    ('8otl3781'      'h9asb4nt'    'opf1qwgl')
1       1    ('opf1qwgl'      'bsgucexp'    '7id8tonh')
2       1    ('vif0h7f9'      'vfoxp8tl'    '8otl3781')
3       1    ('qs83or5w'      '1o9oqnd6'    'tded20ih')
4       1    ('h9asb4nt'      'opf1qwgl'    'bsgucexp')
5       1    ('ic3tdx59'      'qs83or5w'    '1o9oqnd6')
6       1    ('tded20ih'      'vif0h7f9'    'vfoxp8tl')
7       1    ('vfoxp8tl'      '8otl3781'    'h9asb4nt')
8       1    ('1o9oqnd6'      'tded20ih'    'vif0h7f9')


127.0.0.1 - - [22/Jun/2021 19:59:29] "POST /_dash-update-component HTTP/1.1" 200 -



Total: 520

{'af': 1, 'ca': 1, 'de': 2, 'en': 504, 'es': 5, 'fr': 5, 'ro': 1, 'sv': 1}
    values  First keywords middle keywords   last keywoprds
0        7       ('severe'   'respiratory'      'syndrome')
1        4       ('impact'       'covid19'      'pandemic')
2        4  ('coronavirus'       'disease'       'covid19')
3        4  ('respiratory'      'syndrome'   'coronavirus')
4        3  ('angiotensin'    'converting'        'enzyme')
5        3  ('respiratory'     'syncytial'         'virus')
6        3   ('infectious'    'bronchitis'         'virus')
7        3  ('coronavirus'       'disease'      'pandemic')
8        3   ('systematic'        'review'  'metaanalysis')
9        2      ('covid19'        'social'    'distancing')
10       2         ('amid'       'covid19'      'pandemic')
11       2      ('therapy'    'critically'           'ill')


127.0.0.1 - - [22/Jun/2021 19:59:47] "POST /_dash-update-component HTTP/1.1" 200 -


In [ ]:
#
#meta_df = pd.read_csv('metadata.csv', low_memory=False)
#meta_df.head()
#

In [ ]:

#uploade any file from system 
#import ipywidgets as widgets
##import ipython_blocking
#uploader=widgets.FileUpload(
#    accept='*.csv',
#    multiple=False
#)
#display(uploader)


In [ ]:
# %block uploader

In [ ]:

#[input_file]=uploader.value
#meta_df=pd.read_csv(input_file,low_memory=False)
#meta_df.head()


In [ ]:
#meta_df = pd.read_csv('metadata.csv', low_memory=False)
#meta_df.head()

In [ ]:

#output=widgets.Output()
#ALL='ALL'
#dropdown_rows=widgets.Dropdown(options=pd.read_csv(input_file,nrows=0))
#display(dropdown_rows)


In [ ]:
# %block dropdown_rows

In [ ]:
#df_covid = pd.DataFrame(columns=['abstract'])
#df_covid['abstract'] = df.abstract
#df_covid

In [ ]:
#df_covid.info()

In [ ]:
#df_covid.drop_duplicates(['abstract'], inplace=True)
#df_covid.dropna(inplace=True)
'''df_covid.info()'''

In [ ]:
#from langdetect import DetectorFactory, detect

In [ ]:
#Dropping non-English articles

'''DetectorFactory.seed = 0

# hold label - language
languages = []

# go through each text
for ii in tqdm(range(0,len(df_covid))):
    # split by space into list, take the first x intex, join with space
    text = df_covid.iloc[ii]['abstract'].split(" ")
    
    lang = "en"
    try:
        if len(text) > 50:
            lang = detect(" ".join(text[:50]))
        elif len(text) > 0:
            lang = detect(" ".join(text[:len(text)]))
    # ught... beginning of the document was not in a good format
    except Exception as e:
        all_words = set(text)
        try:
            lang = detect(" ".join(all_words))
        
        except Exception as e:        
            lang = "unknown"
            pass
    
    # get the language    
    languages.append(lang)'''

In [ ]:
# Number of articles in each language

'''languages_dict = {}
for lang in set(languages):
    languages_dict[lang] = languages.count(lang)
    
print("Total: {}\n".format(len(languages)))
pprint(languages_dict)'''

In [ ]:
# Dropping non english articles

'''df_covid['language'] = languages
df_covid = df_covid[df_covid['language'] == 'en'] 
df_covid.info()'''

In [ ]:
'''df_covid = df_covid.drop(['language'], axis = 1) 
df_covid.head()'''

In [ ]:
#df_covid.to_csv('modified.csv')

In [ ]:
#df_covid.info()

In [ ]:
#dataset = pd.read_csv('modified.csv')
#dataset.head()

In [ ]:
#dataset.columns = ['label', 'abstract']
#dataset.head()

In [ ]:
#dataset = dataset.drop(columns = 'label')
#dataset.head()

In [ ]:
'''def remove_punctuation(txt):
    txt_nopunct = "".join([c for c in txt if c not in string.punctuation])
    return txt_nopunct'''

In [ ]:
#dataset['abstract_clean'] = dataset['abstract'].apply(lambda x: remove_punctuation(x))
#dataset.head()

In [ ]:
'''import re
def tokenize(txt):
    tokens = re.split('\W+', txt)
    return tokens'''

In [ ]:
'''dataset['abstract_clean_tokenized'] = dataset['abstract_clean'].apply(lambda x: tokenize(x.lower()))
dataset.head()'''

In [ ]:
'''dataset = dataset.drop(columns = ['abstract'])
dataset = dataset.drop(columns = ['abstract_clean'])
dataset.head()'''

In [ ]:
'''# Lemmatization
def lemmatization(token_text):
    text = [wn.lemmatize(word) for word in token_text]
    return text'''

In [ ]:
'''dataset['abs_cln_lemm'] = dataset['abstract_clean_tokenized'].apply(lambda x : lemmatization(x))
dataset.head()'''

In [ ]:
'''def remove_stopwords(txt_tokenized):
    txt_clean = [word for word in txt_tokenized if word not in stopwords]
    return txt_clean

#Remove stop words
import nltk
stopwords = nltk.corpus.stopwords.words('english')

dataset['abs_no_sw'] = dataset['abs_cln_lemm'].apply(lambda x: remove_stopwords(x))
dataset.head()

dataset = dataset.drop(columns = ['abstract_clean_tokenized'])
dataset = dataset.drop(columns = ['abs_cln_lemm'])
dataset.head()

dataset.to_csv('modified_cleaned.csv')

# Custom word removal
new_words = ('2019','95','ci','patient', 'mental', 'health', 'risk', 'factor', 'formula', 'see', 'text', 'polymerase', 'chain', 'reaction' ,'supplementary',
             'material', 'available' ,'intensive', 'care', 'unit', 'contains', 'supplementary', 
            'ha', 'acute', 'study', 'wa', '2', '19', 'white', 'blood', 'cell', 'middle', 'east',
             'aim', 'sars', 'cov', 'electronic', 'online', 'version', 'article',
             'access', 'service', 'among', 'waste', 'collector', 'routine', 'treatment', 'combined', 'chinese', 'herbal', 
            'liaison', 'sarscov2', 's1s2', 'igg', 'assay', 'mechanical', 'ventilation', 's1', 'n', 'p', '0001')

def custom_word_removal(txt_tokenized):
    txt_clean = [word for word in txt_tokenized if word not in new_words]
    return txt_clean

dataset['abstract_keywords'] = dataset['abs_no_sw'].apply(lambda x: custom_word_removal(x))
dataset.head()

dataset = dataset.drop(columns = ['abs_no_sw'])
dataset.head()

dataset.to_csv('abstract_keywords.csv')

df_ngram = pd.read_csv('abstract_keywords.csv', usecols=[1])

df_ngram

def basic_clean(text):

  text = (unicodedata.normalize('NFKD', text)
    .encode('ascii', 'ignore')
    .decode('utf-8', 'ignore')
    .lower())
  words = re.sub(r'[^\w\s]', '', text).split()
  return words

words = basic_clean(''.join(str(df_ngram['abstract_keywords'].tolist())))

words[:20]

(pd.Series(nltk.ngrams(words, 2)).value_counts())[:12]'''

In [ ]:
#(pd.Series(nltk.ngrams(words, 3)).value_counts())[:12]

In [ ]:
#(pd.Series(nltk.ngrams(words, 4)).value_counts())[:12]

In [ ]:
#(pd.Series(nltk.ngrams(words, 5)).value_counts())[:12]

In [ ]:
#(pd.Series(nltk.ngrams(words, 6)).value_counts())[:12]

In [ ]:
#(pd.Series(nltk.ngrams(words, 7)).value_counts())[:12]

In [ ]:
#(pd.Series(nltk.ngrams(words, 8)).value_counts())[:12]

In [ ]:
# bigrams_series = (pd.Series(nltk.ngrams(words, 2)).value_counts())[:12]

In [ ]:
#bigrams_series.to_csv('ram.csv')

In [ ]:
#bigrams_series.sort_values().plot.barh(color='blue', width=.9, figsize=(12, 8))

In [ ]:
#trigrams_series = (pd.Series(nltk.ngrams(words, 3)).value_counts())[:12]

In [ ]:
#trigrams_series.to_csv('tram.csv')

In [ ]:
'''data=pd.read_csv('tram.csv')
data.columns=['keywords','values']
data.dropna(inplace = True)
new= data["keywords"].str.split(", ", n = 2, expand = True)
data["First_keywords"]= new[0]
data["middle_keywords"]= new[1]
data["last_keywoprds"]= new[2]
data.drop(columns =["keywords"], inplace = True)
print (data)
data.to_csv('new.csv')'''

In [ ]:
'''df_data=pd.read_csv('new.csv')
plot_output = widgets.Output()
output_first = widgets.Output()
output=widgets.Output()

ALL='ALL'
def colour_ge_value(value, comparison):
    if value >= comparison:
        return 'color: red'
    else:
        return 'color: black'
def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique
dropdown_first = widgets.Dropdown(options = unique_sorted_values_plus_ALL(df_data.First_keywords))
#dropdown_middle = widgets.Dropdown(options = unique_sorted_values_plus_ALL(df_data.middle_keywords))
bounded_num = widgets.BoundedFloatText(min=0, max=100000, value=5, step=1, description='Number:')

def common_filtering(first, num):
    output.clear_output()
    plot_output.clear_output()
    if(first == ALL):
        common_filter = df_data
    
    else:
        common_filter = df_data[(df_data.First_keywords == first)]
    with output:
        display(common_filter.style.applymap(lambda x: colour_ge_value(x, num), subset=['values']))
    
    common_filter.to_csv("updated.csv")
    df_new=pd.read_csv('updated.csv')
    
    with plot_output:
        fig=px.sunburst(df_new,path=['First_keywords','middle_keywords','last_keywoprds'],values="values")
        fig.show()
        #sns.kdeplot(common_filter['values'], shade=True)
        #plt.show()
    
    #fig=px.sunburst(df_new,path=['First_keywords','middle_keywords','last_keywoprds'],values="values")
    #fig.show()
    
def dropdown_first_eventhandler(change):
    common_filtering(change.new,bounded_num.value)

#def dropdown_middle_eventhandler(change):
#    common_filtering(dropdown_first.value, change.new,bounded_num.value)
    
def bounded_num_eventhandler(change):
    common_filtering(dropdown_first.value,change.new)
        
dropdown_first.observe(dropdown_first_eventhandler, names='value')
#dropdown_middle.observe(dropdown_middle_eventhandler, names='value')
bounded_num.observe(bounded_num_eventhandler, names='value')

display(dropdown_first)
#display(dropdown_middle)
display(bounded_num)

display(output)
display(plot_output)




#fig=px.sunburst(df_new,path=['First keywords','middle keywords','last keywoprds'],values="values")
#fig.show()'''